<a href="https://colab.research.google.com/github/srzarin/AtoZgeospatialsolutions/blob/master/YOLOv5m_TestCustom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How to Test YOLOv5 on Custom Objects

This tutorial is based on the [YOLOv5 repository](https://github.com/ultralytics/yolov5) by [Ultralytics](https://www.ultralytics.com/). This notebook shows testing on **your own custom objects**.
### Steps Covered in this Tutorial

In this tutorial, we will walk through the steps required to test YOLOv5 on your custom objects. You can also use this notebook on your own data.

To test our detector we take the following steps:

* Install YOLOv5 dependencies
* Download custom YOLOv5 object detection data
* Run YOLOv5 inference on test images
* Export saved YOLOv5 output for future inference


#Install Dependencies

_(Remember to choose GPU in Runtime if not already selected. Runtime --> Change Runtime Type --> Hardware accelerator --> GPU)_

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone repo
!pip install -qr yolov5/requirements.txt  # install dependencies (ignore errors)
%cd yolov5

import torch
from IPython.display import Image, clear_output  # to display images
from utils.google_utils import gdrive_download  # to download models/datasets

clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

Setup complete. Using torch 1.6.0+cu101 _CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15079MB, multi_processor_count=40)


# Custom Dataset 

Use the "**YOLOv5 PyTorch**" export format. Note that the Ultralytics implementation calls for a YAML file defining where your test data is.

In [ ]:
# mounting google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# copying train and test zip files from google drive
%cp /content/gdrive/My\ Drive/yolov5/data/test.zip /content

In [ ]:
# unzip the datasets and their contents
!unzip /content/test.zip -d /content/

Archive:  /content/test.zip
   creating: /content/test/
   creating: /content/test/images/
  inflating: /content/test/images/wh_dresscode_0001.jpg  
  inflating: /content/test/images/wh_dresscode_0002.jpg  
  inflating: /content/test/images/wh_dresscode_0003.jpg  
  inflating: /content/test/images/wh_dresscode_0004.jpg  
  inflating: /content/test/images/wh_dresscode_0005.jpg  
  inflating: /content/test/images/wh_dresscode_0006.jpg  
  inflating: /content/test/images/wh_dresscode_0007.jpg  
  inflating: /content/test/images/wh_dresscode_0008.jpg  
  inflating: /content/test/images/wh_dresscode_0052.jpg  
  inflating: /content/test/images/wh_dresscode_0782.jpg  
  inflating: /content/test/images/wh_dresscode_0783.jpg  
  inflating: /content/test/images/wh_dresscode_0784.jpg  
  inflating: /content/test/images/wh_dresscode_0785.jpg  
  inflating: /content/test/images/wh_dresscode_0786.jpg  
  inflating: /content/test/images/wh_dresscode_0787.jpg  
  inflating: /content/test/images/wh_dr

In [ ]:
# copying data.yaml file from google drive
%cp /content/gdrive/My\ Drive/yolov5/data/data.yaml /content

In [ ]:
# copying .mp4 file from google drive for testing
%mkdir /content/videos/
%cp /content/gdrive/My\ Drive/yolov5/data/videos/video1.mp4 /content/videos

In [ ]:
# this is the YAML file
%cd /content
%cat data.yaml

/content
train: ../train/images
val: ../test/images

nc: 5
names: ['hat','vest','goggles','glove','shoe']



# Define Model Configuration and Architecture

We will write a yaml script that defines the parameters for our model like the number of classes, anchors, and each layer.

You do not need to edit these cells, but you may.

In [ ]:
# define number of classes based on YAML
import yaml
with open("data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

In [ ]:
#this is the model configuration we will use for our tutorial 
%cat /content/yolov5/models/yolov5m.yaml

# parameters
nc: 80  # number of classes
depth_multiple: 0.67  # model depth multiple
width_multiple: 0.75  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, BottleneckCSP, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, BottleneckCSP, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-

In [ ]:
#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))


In [ ]:
%%writetemplate /content/yolov5/models/custom_yolov5m.yaml

# parameters
nc: {num_classes}  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, BottleneckCSP, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, BottleneckCSP, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, BottleneckCSP, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, BottleneckCSP, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, BottleneckCSP, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

#Run Inference  With Trained Weights
Run inference with a pretrained checkpoint on contents of `test/images` folder downloaded.

In [ ]:
%cp /content/gdrive/My\ Drive/yolov5/weights/best.pt /content/yolov5/weights/

In [ ]:
# when we ran this, use the best weights!
%cd /content/yolov5/
#!python detect.py --weights runs/exp0_yolov5m_results/weights/best.pt --img 600 --conf 0.4 --source ../test/images
!python detect.py --weights 'weights/best.pt' --img 600 --conf 0.4 --source ../test/images --output inference/output/images

/content/yolov5
Namespace(agnostic_nms=False, augment=False, classes=None, conf_thres=0.4, device='', img_size=600, iou_thres=0.5, output='inference/output/images', save_txt=False, source='../test/images', update=False, view_img=False, weights=['weights/best.pt'])
Using CUDA device0 _CudaDeviceProperties(name='Tesla T4', total_memory=15079MB)

Fusing layers... 
Model Summary: 191 layers, 7.26588e+06 parameters, 0 gradients
image 1/1059 /content/test/images/wh_dresscode_0001.jpg: 480x608 1 hats, Done. (0.049s)
image 2/1059 /content/test/images/wh_dresscode_0002.jpg: 480x608 2 vests, Done. (0.012s)
image 3/1059 /content/test/images/wh_dresscode_0003.jpg: 480x608 1 hats, 1 goggless, Done. (0.012s)
image 4/1059 /content/test/images/wh_dresscode_0004.jpg: 480x608 1 hats, 1 vests, Done. (0.011s)
image 5/1059 /content/test/images/wh_dresscode_0005.jpg: 480x608 3 hats, 3 gloves, Done. (0.013s)
image 6/1059 /content/test/images/wh_dresscode_0006.jpg: 480x608 2 hats, 1 vests, Done. (0.014s)
imag

In [ ]:
# when we ran this, use the best weights!
%cd /content/yolov5/
!python detect.py --weights 'weights/best.pt' --source ../videos/video1.mp4 --output inference/output/videos

!python detect.py --weights ./weights/best.pt --img 416 --conf 0.4 --source ./inference/images/


!python detect.py --weights ./weights/best.pt --source ./inference/images/Vid.mp4 --output ./inference/output/videos

Streaming output truncated to the last 5000 lines.
video 1/1 (370/5372) /content/videos/video1.mp4: 384x640 Done. (0.011s)
video 1/1 (371/5372) /content/videos/video1.mp4: 384x640 Done. (0.011s)
video 1/1 (372/5372) /content/videos/video1.mp4: 384x640 Done. (0.011s)
video 1/1 (373/5372) /content/videos/video1.mp4: 384x640 Done. (0.011s)
video 1/1 (374/5372) /content/videos/video1.mp4: 384x640 Done. (0.010s)
video 1/1 (375/5372) /content/videos/video1.mp4: 384x640 Done. (0.013s)
video 1/1 (376/5372) /content/videos/video1.mp4: 384x640 Done. (0.011s)
video 1/1 (377/5372) /content/videos/video1.mp4: 384x640 Done. (0.013s)
video 1/1 (378/5372) /content/videos/video1.mp4: 384x640 Done. (0.014s)
video 1/1 (379/5372) /content/videos/video1.mp4: 384x640 Done. (0.011s)
video 1/1 (380/5372) /content/videos/video1.mp4: 384x640 Done. (0.011s)
video 1/1 (381/5372) /content/videos/video1.mp4: 384x640 Done. (0.011s)
video 1/1 (382/5372) /content/videos/video1.mp4: 384x640 Done. (0.013s)
video 1/1 (38

# Export Output for Future Inference

Now that you have trained your custom detector, you can export the output for inference on your device elsewhere

In [ ]:
%cp -r /content/yolov5/inference/output/videos /content/gdrive/My\ Drive/yolov5/output/videos
#%cp /content/yolov5/inference/output/videos/video9.mp4 /content/gdrive/My\ Drive/yolov5/output/videos

In [ ]:
%cp -r /content/yolov5/inference/output/images /content/gdrive/My\ Drive/yolov5/output/images
#%cp /content/yolov5/inference/output/images/*.jpg /content/gdrive/My\ Drive/yolov5/output/images

## Thank you!